<a href="https://colab.research.google.com/github/khaled-kk/Image-Processing-Attack/blob/main/IPM2S23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage
from skimage.metrics import structural_similarity as compare_ssim
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy
from PIL import Image
import concurrent.futures
from functools import partial

In [ ]:
def get_label_mapping(mapping_file):
    mapping = {}
    with open(mapping_file) as f:
        for line in f:
            line = line.strip()
            if line:
                id_, label = line.split()
                mapping[id_] = label
    return mapping

def get_ground_truth(ground_truth_file):
    with open(ground_truth_file) as f:
        ground_truth = [int(line.strip()) for line in f]
    return ground_truth

def get_labels(labels_file):
    labels = {}
    with open(labels_file) as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split(None, 1)
                id_ = parts[0]
                label = parts[1] if len(parts) > 1 else ''
                labels[id_] = label
    return labels



In [ ]:
mapping_labels = get_label_mapping('ILSVRC2012_mapping.txt')
GT = get_ground_truth('ILSVRC2012_validation_ground_truth.txt')

In [ ]:
mapperGT = []
for item in GT:
    mapperGT.append(mapping_labels[str(item)])

In [ ]:
def get_class_name(ground_truth_mapping, label_file_path):
    # Load the label file
    with open(label_file_path) as f:
        label_map = {}
        for line in f:
            key, value = line.strip().split('\t')
            label_map[key] = value

    # Map the ground truth using the label map
    labels = []
    for item in ground_truth_mapping:
        label = label_map.get(item, None)
        if label is not None:
            labels.append(label)

    return labels

In [ ]:
ground_truth_class = get_class_name(mapperGT,'labels.txt')

In [ ]:
class_labels = []

with open('ImageNetLabels.txt', 'r') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespaces
        if line:  # Skip empty lines
            label = line.split('\t')[1]  # Extract the label
            class_labels.append(label)


In [ ]:
indices = []

for item in ground_truth_class:
    try:
        index = class_labels.index(item)
        indices.append(index)
    except ValueError:
        indices.append(0)  # Append None if the item is not found in labels

In [ ]:
def fgsm_attack(image, epsilon, model, target_class_index):
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, axis=0)  # Add extra dimension
    target = tf.zeros((1, 1000))  # Target tensor with shape (1, 1000)
    indices = tf.constant([[0, target_class_index]])  # Indices to update
    updates = tf.constant([1.0])  # Values to update with
    target = tf.tensor_scatter_nd_update(target, indices, updates)  # Update target tensor
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.CategoricalCrossentropy()(target, prediction)
    gradient = tape.gradient(loss, image)
    signed_gradient = tf.sign(gradient)
    perturbed_image = image + epsilon * signed_gradient
    perturbed_image = tf.clip_by_value(perturbed_image, 0, 255)
    perturbed_image = tf.squeeze(perturbed_image, axis=0)  # Remove extra dimension
    return perturbed_image

def get_target_class_index(image_path):
    digits =image_path[-9:-5]  # Extracts the last 4 digits before .JPEG
    target_class_index = indices[int(digits)-1]
    return target_class_index

def process_image(image_path, model, epsilon, output_dir):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = np.array(img)
    img = preprocess_input(img)
    target_class_index = get_target_class_index(image_path)
    perturbed_img = fgsm_attack(img, epsilon, model, target_class_index)

    # Convert TensorFlow tensor to PIL image
    perturbed_img = tf.keras.preprocessing.image.array_to_img(perturbed_img)

    # Save perturbed image
    output_filename = os.path.basename(image_path)
    output_path = os.path.join(output_dir, output_filename)
    perturbed_img.save(output_path)
    return perturbed_img


# Example usage
model_name = 'InceptionV3'
image_dir = '/content/drive/MyDrive/Sorted QF folders/Original'
epsilon = 0  # Epsilon value for FGSM attack change
batch_size = 32  # Number of images to process in parallel

# Load the model
if model_name == 'InceptionV3':
    model = InceptionV3(weights='imagenet', include_top=True)
    img_width, img_height = 299, 299
else:
    raise ValueError('Invalid model name')

# Get the list of image filenames
image_filenames = [filename for filename in os.listdir(image_dir) if '\0' not in filename]

#CHANGE output_dir
output_dir = '/content/drive/MyDrive/FGSM/0/NoDefence/'  # Directory to save perturbed images

# Create the output folder if it doesn't exist
os.makedirs(os.path.dirname(output_dir), exist_ok=True)

processed_files = [filename for filename in os.listdir(output_dir) if filename.endswith('.JPEG')]
image_filenames = [filename for filename in image_filenames if filename not in processed_files]

# Process images in parallel using multithreading
with concurrent.futures.ThreadPoolExecutor() as executor:
    process_func = partial(process_image, model=model, epsilon=epsilon, output_dir=output_dir)
    executor.map(process_func, [os.path.join(image_dir, filename) for filename in image_filenames], chunksize=batch_size)


In [ ]:
import os
import tensorflow as tf

input_folder = "/content/drive/MyDrive/FGSM/0/NoDefence"#you need to change the path for each Epsilon
output_folder = "/content/drive/MyDrive/FGSM/0/QF-"
qf_values = range(10, 101, 10)

# Loop through the input folder and process the first 2000 images
for i, filename in enumerate(os.listdir(input_folder)):
    if i >= 1000:
        break
    if filename.endswith(".JPEG"):
        # Load the image using TensorFlow
        image_path = os.path.join(input_folder, filename)
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)  # RGB image

        # Loop through the QF values and compress the image at each quality level
        for qf in qf_values:
            # Set the output filename based on the original filename and the QF value
            output_filename = f"{filename[:-4]}_qf{qf}.jpg"
            output_path = os.path.join(f"{output_folder}{qf}/", output_filename)

            # Convert the image tensor to JPEG format with the specified quality
            compressed_image = tf.image.encode_jpeg(image, format='rgb', quality=qf)

            # Create the output folder if it doesn't exist
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            # Save the compressed image to the output path
            tf.io.write_file(output_path, compressed_image)


In [ ]:
def pgd_attack(image, epsilon, model, target_class_index, num_steps, step_size):
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, axis=0)  # Add extra dimension
    target = tf.zeros((1, 1000))  # Target tensor with shape (1, 1000)
    indices = tf.constant([[0, target_class_index]])  # Indices to update
    updates = tf.constant([1.0])  # Values to update with
    target = tf.tensor_scatter_nd_update(target, indices, updates)  # Update target tensor

    for _ in range(num_steps):
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.CategoricalCrossentropy()(target, prediction)
        gradient = tape.gradient(loss, image)
        signed_gradient = tf.sign(gradient)
        perturbed_image = image + step_size * signed_gradient
        perturbed_image = tf.clip_by_value(perturbed_image, image - epsilon, image + epsilon)
        perturbed_image = tf.clip_by_value(perturbed_image, 0, 255)
        image = perturbed_image

    perturbed_image = tf.squeeze(image, axis=0)  # Remove extra dimension
    return perturbed_image


def get_target_class_index(image_path):
#     print(image_path)
    digits =image_path[-9:-5]  # Extracts the last 4 digits before .JPEG
    target_class_index = indices[int(digits)-1]
    return target_class_index

def process_image(image_path, model, epsilon, output_dir):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = np.array(img)
    img = preprocess_input(img)
    target_class_index = get_target_class_index(image_path)
#     print(target_class_index)

    num_steps = 10
    step_size = 0.03
    perturbed_img = pgd_attack(img, epsilon, model, target_class_index, num_steps, step_size)

    # Convert TensorFlow tensor to PIL image
    perturbed_img = tf.keras.preprocessing.image.array_to_img(perturbed_img)
    print(perturbed_img)
    # Save perturbed image
    output_filename = os.path.basename(image_path)
    output_path = os.path.join(output_dir, output_filename)
    perturbed_img.save(output_path)
    return perturbed_img

# Example usage
model_name = 'InceptionV3'
image_dir = '/content/drive/MyDrive/Sorted QF folders/Original'
epsilon = 2  # Epsilon value for FGSM attack
batch_size = 32  # Number of images to process in parallel

# Load the model
if model_name == 'InceptionV3':
    model = InceptionV3(weights='imagenet', include_top=True)
    img_width, img_height = 299, 299
else:
    raise ValueError('Invalid model name')

# Get the list of image filenames
image_filenames = [filename for filename in os.listdir(image_dir) if '\0' not in filename]

#CHANGE output_dir
output_dir = '/content/drive/MyDrive/pgd/2/NoDefence/'  # Directory to save perturbed images

processed_files = [filename for filename in os.listdir(output_dir) if filename.endswith('.JPEG')]
image_filenames = [filename for filename in image_filenames if filename not in processed_files]

# Process images in parallel using multithreading
with concurrent.futures.ThreadPoolExecutor() as executor:
    process_func = partial(process_image, model=model, epsilon=epsilon, output_dir=output_dir)
    executor.map(process_func, [os.path.join(image_dir, filename) for filename in image_filenames], chunksize=batch_size)


<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19C0BF2650>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BC496CE0>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BD108B20>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BCE90460>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BC379C90>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BCE91960>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BD691FC0>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BD0EF760>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BC37A890>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BD1095D0>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BD0BE950>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BC495A20>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BD690DF0>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BC494880>
<PIL.Image.Image image mode=RGB size=299x299 at 0x7C19BD0BDCC0>
<PIL.Image.Image image mode=RGB size=299

In [ ]:
import os
import tensorflow as tf

input_folder = "/content/drive/MyDrive/pgd/2/NoDefence"
output_folder = "/content/drive/MyDrive/pgd/2/QF-"
qf_values = range(10, 101, 10)

# Loop through the input folder and process the first 2000 images
for i, filename in enumerate(os.listdir(input_folder)):
    if i >= 1000:
        break
    if filename.endswith(".JPEG"):
        # Load the image using TensorFlow
        image_path = os.path.join(input_folder, filename)
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)  # RGB image

        # Loop through the QF values and compress the image at each quality level
        for qf in qf_values:
            # Set the output filename based on the original filename and the QF value
            output_filename = f"{filename[:-4]}_qf{qf}.jpg"
            output_path = os.path.join(f"{output_folder}{qf}/", output_filename)

            # Convert the image tensor to JPEG format with the specified quality
            compressed_image = tf.image.encode_jpeg(image, format='rgb', quality=qf)

            # Create the output folder if it doesn't exist
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            # Save the compressed image to the output path
            tf.io.write_file(output_path, compressed_image)


In [ ]:
# Load the Inception V3 model
model = InceptionV3(weights='imagenet', include_top=True)

In [ ]:
def predict_images(model_name, image_dir,QF):
    # Load the model
    if model_name == 'InceptionV3':
        model = InceptionV3(weights='imagenet',include_top=True)
        img_width, img_height = 299, 299
    else:
        raise ValueError('Invalid model name')

    # Set up the data generator
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Get the list of image filenames
    image_filenames = sorted(os.listdir(image_dir))
    # Create a generator for the images in batches
    def image_batch_generator(filenames, batch_size):
        for i in range(0, len(filenames), batch_size):
            batch_filenames = filenames[i:i+batch_size]
            batch_images = []
            for filename in batch_filenames:
                image_path = os.path.join(image_dir, filename)
                img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
                img = np.array(img)
                img = preprocess_input(img)
                batch_images.append(img)
            yield np.array(batch_images)



    # Predict the classes of the images in batches
    predictions = []
    for batch_images in image_batch_generator(image_filenames, batch_size=32):
        batch_predictions = model.predict(batch_images)
        predictions.append(batch_predictions)
    predictions = np.concatenate(predictions)
    decoded_preds = decode_predictions(predictions, top=5)

    predsTop5Index=[]
    predsTop1Index=[]
    for i in range(len(decoded_preds)):
        top5 = []
        class_index, class_name, confidence = decoded_preds[i][0]
        predsTop1Index.append((class_index, class_name))
        for j in range(5):
            class_index, class_name, confidence = decoded_preds[i][j]
            top5.append((class_index,class_name))
        predsTop5Index.append(top5)
    with open(f"Predictions-NoAttack-{model_name}-Prediction-{QF}.txt", 'w') as f:
        for i in range(len(predsTop1Index)):
            predicted_class,predicted_label =predsTop1Index[i]
            f.write(f'{predicted_class} {predicted_label} \n')

    with open(f"Predictions-NoAttack-{model_name}-PredictionTOP5-{QF}.txt", 'w') as f:
        for i in range(len(predsTop5Index)):
            f.write(f'{predsTop5Index[i]} \n')

In [ ]:
# specify the path to the main folder
main_folder = '/content/drive/MyDrive/Sorted QF folders'
# loop through all the files in the main folder and its subfolders
for root, dirs, files in os.walk(main_folder):
    for file in dirs:
        # do something with the file
        path = os.path.join(root, file)
        print(path)
        predict_images('InceptionV3',path,file)

/content/drive/MyDrive/Sorted QF folders/QF-20
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/Sorted QF folders/QF-30
1/1 [==============================] - 5s 5s/step
/content/drive/MyDrive/Sorted QF folders/QF-40
1/1 [==============================] - 4s 4s/step
/content/drive/MyDrive/Sorted QF folders/QF-50
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/Sorted QF folders/QF-60
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/Sorted QF folders/QF-70
1/1 [==============================] - 4s 4s/step
/content/drive/MyDrive/Sorted QF folders/QF-80
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/Sorted QF folders/QF-90
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/Sorted QF folders/QF-10
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/Sorted QF folders/QF-100
1/1 [==============================] - 5s 5s/step
/content/drive/MyDrive/Sorted

In [ ]:
def predict_images(model_name, image_dir,QF):
    # Load the model
    if model_name == 'InceptionV3':
        model = InceptionV3(weights='imagenet',include_top=True)
        img_width, img_height = 299, 299
    else:
        raise ValueError('Invalid model name')

    # Set up the data generator
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Get the list of image filenames
    image_filenames = sorted(os.listdir(image_dir))
    # Create a generator for the images in batches
    def image_batch_generator(filenames, batch_size):
        for i in range(0, len(filenames), batch_size):
            batch_filenames = filenames[i:i+batch_size]
            batch_images = []
            for filename in batch_filenames:
                image_path = os.path.join(image_dir, filename)
                img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
                img = np.array(img)
                img = preprocess_input(img)
                batch_images.append(img)
            yield np.array(batch_images)



    # Predict the classes of the images in batches
    predictions = []
    for batch_images in image_batch_generator(image_filenames, batch_size=32):
        batch_predictions = model.predict(batch_images)
        predictions.append(batch_predictions)
    predictions = np.concatenate(predictions)
    decoded_preds = decode_predictions(predictions, top=5)

    predsTop5Index=[]
    predsTop1Index=[]
    for i in range(len(decoded_preds)):
        top5 = []
        class_index, class_name, confidence = decoded_preds[i][0]
        predsTop1Index.append((class_index, class_name))
        for j in range(5):
            class_index, class_name, confidence = decoded_preds[i][j]
            top5.append((class_index,class_name))
        predsTop5Index.append(top5)
    with open(f"Predictions-0-FGSM-{model_name}-Prediction-{QF}.txt", 'w') as f:
        for i in range(len(predsTop1Index)):
            predicted_class,predicted_label =predsTop1Index[i]
            f.write(f'{predicted_class} {predicted_label} \n')

    with open(f"Predictions-0-FGSM-{model_name}-PredictionTOP5-{QF}.txt", 'w') as f:
        for i in range(len(predsTop5Index)):
            f.write(f'{predsTop5Index[i]} \n')

In [ ]:
# specify the path to the main folder
main_folder = '/content/drive/MyDrive/FGSM/0/'
# loop through all the files in the main folder and its subfolders
for root, dirs, files in os.walk(main_folder):
    for file in dirs:
        # do something with the file
        path = os.path.join(root, file)
        print(path)
        predict_images('InceptionV3',path,file)

/content/drive/MyDrive/FGSM/0/NoDefence
1/1 [==============================] - 8s 8s/step
/content/drive/MyDrive/FGSM/0/QF-10
1/1 [==============================] - 6s 6s/step
/content/drive/MyDrive/FGSM/0/QF-20
1/1 [==============================] - 6s 6s/step
/content/drive/MyDrive/FGSM/0/QF-30
1/1 [==============================] - 6s 6s/step
/content/drive/MyDrive/FGSM/0/QF-40
1/1 [==============================] - 7s 7s/step
/content/drive/MyDrive/FGSM/0/QF-50
1/1 [==============================] - 7s 7s/step
/content/drive/MyDrive/FGSM/0/QF-60
1/1 [==============================] - 6s 6s/step
/content/drive/MyDrive/FGSM/0/QF-70
1/1 [==============================] - 8s 8s/step
/content/drive/MyDrive/FGSM/0/QF-80
1/1 [==============================] - 6s 6s/step
/content/drive/MyDrive/FGSM/0/QF-90
1/1 [==============================] - 6s 6s/step
/content/drive/MyDrive/FGSM/0/QF-100
1/1 [==============================] - 6s 6s/step


In [ ]:
def predict_images(model_name, image_dir,QF):
    # Load the model
    if model_name == 'InceptionV3':
        model = InceptionV3(weights='imagenet',include_top=True)
        img_width, img_height = 299, 299
    else:
        raise ValueError('Invalid model name')

    # Set up the data generator
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Get the list of image filenames
    image_filenames = sorted(os.listdir(image_dir))
    # Create a generator for the images in batches
    def image_batch_generator(filenames, batch_size):
        for i in range(0, len(filenames), batch_size):
            batch_filenames = filenames[i:i+batch_size]
            batch_images = []
            for filename in batch_filenames:
                image_path = os.path.join(image_dir, filename)
                img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
                img = np.array(img)
                img = preprocess_input(img)
                batch_images.append(img)
            yield np.array(batch_images)

    # Predict the classes of the images in batches
    predictions = []
    for batch_images in image_batch_generator(image_filenames, batch_size=32):
        batch_predictions = model.predict(batch_images)
        predictions.append(batch_predictions)
    predictions = np.concatenate(predictions)
    decoded_preds = decode_predictions(predictions, top=5)
    #print(decoded_preds[0][0][0])

    predsTop5Index=[]
    predsTop1Index=[]
    for i in range(len(decoded_preds)):
        top5 = []
        class_index, class_name, confidence = decoded_preds[i][0]
        predsTop1Index.append((class_index, class_name))
        for j in range(5):
            class_index, class_name, confidence = decoded_preds[i][j]
            top5.append((class_index,class_name))
        predsTop5Index.append(top5)
    with open(f"Predictions-2-PGD-{model_name}-Prediction-{QF}.txt", 'w') as f:
        for i in range(len(predsTop1Index)):
            predicted_class,predicted_label =predsTop1Index[i]
            f.write(f'{predicted_class} {predicted_label} \n')

    with open(f"Predictions-2-PGD-{model_name}-PredictionTOP5-{QF}.txt", 'w') as f:
        for i in range(len(predsTop5Index)):
            f.write(f'{predsTop5Index[i]} \n')

In [ ]:
# specify the path to the main folder
main_folder = '/content/drive/MyDrive/pgd/2/'
# loop through all the files in the main folder and its subfolders
for root, dirs, files in os.walk(main_folder):
    for file in dirs:
        # do something with the file
        path = os.path.join(root, file)
        print(path)
        predict_images('InceptionV3',path,file)

/content/drive/MyDrive/pgd/2/NoDefence
35363/35363 [==============================] - 0s 0us/step
/content/drive/MyDrive/pgd/2/QF-10
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/pgd/2/QF-20
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/pgd/2/QF-30
1/1 [==============================] - 4s 4s/step
/content/drive/MyDrive/pgd/2/QF-40
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/pgd/2/QF-50
1/1 [==============================] - 5s 5s/step
/content/drive/MyDrive/pgd/2/QF-60
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/pgd/2/QF-70
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/pgd/2/QF-80
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/pgd/2/QF-90
1/1 [==============================] - 3s 3s/step
/content/drive/MyDrive/pgd/2/QF-100
1/1 [==============================] - 3s 3s/step


In [ ]:
def get_predicted_labels(labels_file):
    labels = []
    with open(labels_file) as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split(None, 2)
                labels.append(parts[0])
    return labels

In [ ]:
def get_predicted_Top5_labels(labels_file):
    labels = []
    with open(labels_file) as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split(None)
                arr = [parts[0],parts[2],parts[4],parts[6],parts[8]]
                outarr = [s.replace("[('", '').replace("',", '').replace("('", '') for s in arr]
                labels.append(outarr)

    return labels

In [ ]:
readlabels=['QF-10','QF-20','QF-30','QF-40','QF-50','QF-60','QF-70','QF-80','QF-90','QF-100','NoDefence']

In [ ]:
QFlabels=['QF-10','QF-20','QF-30','QF-40','QF-50','QF-60','QF-70','QF-80','QF-90','QF-100','Original']

In [ ]:
def findTOP1n(true_labels,labels,model,attack):
    Top1arr =[]
    for file in labels:
        filepath = f'Predictions-{attack}-{model}-Prediction-'+file+'.txt'
        pred= get_predicted_labels(filepath)
        top1_correct = sum([true_label == pred for true_label, pred in zip(true_labels, pred)])
        top1_accuracy = top1_correct / len(pred) *100
        Top1arr.append(file+" : " +str(top1_accuracy))
    return Top1arr

In [ ]:
findTOP1n(mapperGT,QFlabels,'InceptionV3','NoAttack')

['QF-10 : 67.80000000000001',
 'QF-20 : 72.3',
 'QF-30 : 73.5',
 'QF-40 : 75.4',
 'QF-50 : 75.3',
 'QF-60 : 75.7',
 'QF-70 : 75.5',
 'QF-80 : 75.1',
 'QF-90 : 75.0',
 'QF-100 : 75.4',
 'Original : 75.4']

In [ ]:
def findTOP1(true_labels,labels,model,attack,eps):
    Top1arr =[]
    for file in labels:
        filepath = f'Predictions-{eps}-{attack}-{model}-Prediction-'+file+'.txt'
        pred= get_predicted_labels(filepath)
        top1_correct = sum([true_label == pred for true_label, pred in zip(true_labels, pred)])
        top1_accuracy = top1_correct / len(pred) *100
        Top1arr.append(file+" : " +str(top1_accuracy))
    return Top1arr

In [ ]:
findTOP1(mapperGT,readlabels,'InceptionV3','FGSM','0')

['QF-10 : 0.33333333333333337',
 'QF-20 : 0.33333333333333337',
 'QF-30 : 0.33333333333333337',
 'QF-40 : 0.33333333333333337',
 'QF-50 : 0.33333333333333337',
 'QF-60 : 0.5',
 'QF-70 : 0.33333333333333337',
 'QF-80 : 0.33333333333333337',
 'QF-90 : 0.33333333333333337',
 'QF-100 : 0.33333333333333337',
 'NoDefence : 0.33333333333333337']

In [ ]:
findTOP1(mapperGT,readlabels,'InceptionV3','FGSM','2')

['QF-10 : 0.3',
 'QF-20 : 0.3',
 'QF-30 : 0.3',
 'QF-40 : 0.3',
 'QF-50 : 0.3',
 'QF-60 : 0.3',
 'QF-70 : 0.5',
 'QF-80 : 0.5',
 'QF-90 : 0.4',
 'QF-100 : 0.4',
 'NoDefence : 0.5']

In [ ]:
findTOP1(mapperGT,readlabels,'InceptionV3','FGSM','6')

['QF-10 : 0.1',
 'QF-20 : 0.1',
 'QF-30 : 0.0',
 'QF-40 : 0.1',
 'QF-50 : 0.1',
 'QF-60 : 0.0',
 'QF-70 : 0.0',
 'QF-80 : 0.0',
 'QF-90 : 0.0',
 'QF-100 : 0.0',
 'NoDefence : 0.0']

In [ ]:
findTOP1(mapperGT,readlabels,'InceptionV3','PGD','0')

['QF-10 : 37.9',
 'QF-20 : 45.9',
 'QF-30 : 48.699999999999996',
 'QF-40 : 51.0',
 'QF-50 : 48.4',
 'QF-60 : 50.6',
 'QF-70 : 54.0',
 'QF-80 : 54.6',
 'QF-90 : 54.0',
 'QF-100 : 54.7',
 'NoDefence : 56.3']

In [ ]:
findTOP1(mapperGT,readlabels,'InceptionV3','PGD','2')

['QF-10 : 16.7',
 'QF-20 : 7.3999999999999995',
 'QF-30 : 4.1000000000000005',
 'QF-40 : 2.7',
 'QF-50 : 2.5',
 'QF-60 : 1.2',
 'QF-70 : 0.7000000000000001',
 'QF-80 : 0.7000000000000001',
 'QF-90 : 0.7000000000000001',
 'QF-100 : 0.7000000000000001',
 'NoDefence : 0.6']

In [ ]:
findTOP1(mapperGT,readlabels,'InceptionV3','PGD','6')

['QF-10 : 16.0',
 'QF-20 : 8.5',
 'QF-30 : 5.1',
 'QF-40 : 3.3000000000000003',
 'QF-50 : 3.5000000000000004',
 'QF-60 : 1.4000000000000001',
 'QF-70 : 0.7000000000000001',
 'QF-80 : 0.7000000000000001',
 'QF-90 : 0.7000000000000001',
 'QF-100 : 0.7000000000000001',
 'NoDefence : 0.7000000000000001']

In [ ]:
def findTOP5n(true_labels,labels,model,attack):
    Top5arr =[]
    for file in labels:
        filepath = f'Predictions-{attack}-{model}-PredictionTOP5-'+file+'.txt'
        pred= get_predicted_Top5_labels(filepath)
        top5_correct = sum([any(true_label in top5 for top5 in sublist) for sublist, true_label in zip(pred, true_labels)])
        top5_accuracy = top5_correct / len(pred) * 100
        Top5arr.append(file+" : " +str(top5_accuracy))
    return Top5arr

In [ ]:
findTOP5n(mapperGT,QFlabels,'InceptionV3','NoAttack')

['QF-10 : 89.0',
 'QF-20 : 92.0',
 'QF-30 : 92.0',
 'QF-40 : 92.60000000000001',
 'QF-50 : 92.80000000000001',
 'QF-60 : 92.80000000000001',
 'QF-70 : 92.80000000000001',
 'QF-80 : 92.5',
 'QF-90 : 93.0',
 'QF-100 : 92.60000000000001',
 'Original : 92.60000000000001']

In [ ]:
def findTOP5(true_labels,labels,model,attack,eps):
    Top5arr =[]
    for file in labels:
        filepath = f'Predictions-{eps}-{attack}-{model}-PredictionTOP5-'+file+'.txt'
        pred= get_predicted_Top5_labels(filepath)
        top5_correct = sum([any(true_label in top5 for top5 in sublist) for sublist, true_label in zip(pred, true_labels)])
        top5_accuracy = top5_correct / len(pred) * 100
        Top5arr.append(file+" : " +str(top5_accuracy))
    return Top5arr

In [ ]:
findTOP5(mapperGT,readlabels,'InceptionV3','FGSM','0')

['QF-10 : 0.6666666666666667',
 'QF-20 : 0.5',
 'QF-30 : 0.6666666666666667',
 'QF-40 : 0.5',
 'QF-50 : 0.6666666666666667',
 'QF-60 : 1.0',
 'QF-70 : 0.6666666666666667',
 'QF-80 : 0.5',
 'QF-90 : 0.33333333333333337',
 'QF-100 : 0.5',
 'NoDefence : 0.5']

In [ ]:
findTOP5(mapperGT,readlabels,'InceptionV3','FGSM','2')

['QF-10 : 1.7000000000000002',
 'QF-20 : 1.2',
 'QF-30 : 1.0',
 'QF-40 : 1.0',
 'QF-50 : 0.8',
 'QF-60 : 0.8',
 'QF-70 : 1.2',
 'QF-80 : 0.8999999999999999',
 'QF-90 : 0.8999999999999999',
 'QF-100 : 1.0',
 'NoDefence : 1.0']

In [ ]:
findTOP5(mapperGT,readlabels,'InceptionV3','FGSM','6')

['QF-10 : 0.4',
 'QF-20 : 0.3',
 'QF-30 : 0.6',
 'QF-40 : 0.5',
 'QF-50 : 0.6',
 'QF-60 : 0.4',
 'QF-70 : 0.4',
 'QF-80 : 0.6',
 'QF-90 : 0.6',
 'QF-100 : 0.7000000000000001',
 'NoDefence : 0.5']

In [ ]:
findTOP5(mapperGT,readlabels,'InceptionV3','PGD','0')

['QF-10 : 59.4',
 'QF-20 : 68.89999999999999',
 'QF-30 : 71.1',
 'QF-40 : 72.6',
 'QF-50 : 71.5',
 'QF-60 : 73.9',
 'QF-70 : 77.8',
 'QF-80 : 78.10000000000001',
 'QF-90 : 77.5',
 'QF-100 : 78.0',
 'NoDefence : 79.2']

In [ ]:
findTOP5(mapperGT,readlabels,'InceptionV3','PGD','2')

['QF-10 : 34.599999999999994',
 'QF-20 : 24.8',
 'QF-30 : 18.099999999999998',
 'QF-40 : 16.0',
 'QF-50 : 15.1',
 'QF-60 : 7.3',
 'QF-70 : 3.6999999999999997',
 'QF-80 : 2.9000000000000004',
 'QF-90 : 3.2',
 'QF-100 : 2.7',
 'NoDefence : 2.4']

In [ ]:
findTOP5(mapperGT,readlabels,'InceptionV3','PGD','6')

['QF-10 : 35.9',
 'QF-20 : 26.6',
 'QF-30 : 17.8',
 'QF-40 : 16.0',
 'QF-50 : 16.0',
 'QF-60 : 8.5',
 'QF-70 : 3.5999999999999996',
 'QF-80 : 3.2',
 'QF-90 : 3.3000000000000003',
 'QF-100 : 3.1',
 'NoDefence : 3.1']